In [1]:
# Test text API for audioplay tool

import os, re
import torch; torch._dynamo.config.recompile_limit = 64;

In [2]:
from unsloth import FastModel


model, processor = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None,
    max_seq_length = 1024,
    load_in_4bit = True,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/m1/nvme/p311uns2/lib/python3.11/site-packages/unsloth_zoo/gradient_checkpointing.py:355: UserWarning: expandable_segments not supported on this platform (Triggered internally at /pytorch/c10/hip/HIPAllocatorConfig.h:36.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2026.1.4: Fast Gemma3N patching. Transformers: 4.57.6.
   \\   /|    Radeon RX 7900 XT. Num GPUs = 1. Max memory: 19.984 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+rocm6.4. ROCm Toolkit: 6.4.43482-0f2d60242. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(f"Attn Implementation: {model.config._attn_implementation}")

Attn Implementation: sdpa


In [6]:
# функции для инференса

from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template


# Check out available chat templates and how they are applied
# https://github.com/unslothai/unsloth/blob/main/unsloth/chat_templates.py
tokenizer = get_chat_template(processor, chat_template="gemma3n")

# Helper function for inference
def streaming_inference(messages, max_new_tokens = 128):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True,
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        do_sample=False,
        streamer = TextStreamer(processor, skip_prompt = True),
    )

In [7]:
from transformers import TextStreamer
import io
import sys


class CaptureTextStreamer(TextStreamer):
    def __init__(self, tokenizer, skip_prompt=True, **kwargs):
        super().__init__(tokenizer, skip_prompt=skip_prompt, **kwargs)
        self.generated_text = ""
        
    def on_finalized_text(self, text: str, stream_end: bool = False):
        # Сохраняем текст
        self.generated_text += text
        # Также выводим в консоль (опционально)
        print(text, end="", flush=True)

# Модифицированная функция для захвата текста
def capture_inference(messages, max_new_tokens=128):
    # Создаем кастомный streamer
    streamer = CaptureTextStreamer(processor, skip_prompt=True)
    
    # Генерируем ответ
    output = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to("cuda"),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        streamer=streamer,
    )
    
    # Возвращаем захваченный текст
    return streamer.generated_text

In [9]:
from datasets import Dataset, Audio, Features, Value
from pathlib import Path
import pandas as pd

data_path = "audio_assistant_queries.csv"

df = pd.read_csv(
    data_path,
    delimiter=",",
)
df.head(15)

,Request,Response
0,Stop,<tool_call><name>audioplay.play_stop</name><ar...
1,Stop playing,<tool_call><name>audioplay.play_stop</name><ar...
2,Stop music,<tool_call><name>audioplay.play_stop</name><ar...
3,Break music,<tool_call><name>audioplay.play_stop</name><ar...
4,Break playing,<tool_call><name>audioplay.play_stop</name><ar...
5,Stop audio,<tool_call><name>audioplay.play_stop</name><ar...
6,Cancel audio,<tool_call><name>audioplay.play_stop</name><ar...
7,Tell me the playing status.,<tool_call><name>audioplay.play_status</name><...
8,What's the status of playing?,<tool_call><name>audioplay.play_status</name><...
9,status!,<tool_call><name>audioplay.play_status</name><...


In [ ]:
# Создаём Dataset из DataFrame
#ds = Dataset.from_pandas(df, preserve_index=False)

In [11]:
for row in df.head(1).itertuples():
    text_request = row.Request
    text_gtresponse = row.Response
    print(f"{text_request} - {text_gtresponse}")

Stop - <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>


In [13]:
SYSTEM_PROMPT = """You are a tool-using assistant that can understand audio. You are given a user's request in audio format.
You MUST transcribe user's audio and respond with exactly one XML tool call if the tool usage is required (user is asking about play music, asking music title or artist or Genre or Album), otherwise respond to user's request in plain text.
Template: <tool_call><name>audioplay.play_request</name><arguments><artist>ARTIST</artist><title>TITLE</title><genre>GENRE</genre><album>Album</album></arguments></tool_call>. If some arguments is not recognised the correpondence field should be empty. If tool usage is not required respond shortly with text

Also You MUST transcribe user's control command's to stop playing, such as stop, break, cancel to following Template: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

Also You MUST transcribe user's control command's about playing status, such as status (give status) to following Template: <tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>
"""

result =[]

for row in df.head(10).itertuples():
    text_request = row.Request
    text_gtresponse = row.Response
    
    test_messages = [            
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYSTEM_PROMPT,
                }
            ],
        },
        {
            "role": "user",
            "content": [            
                {"type": "text", "text": text_request}
                
            ]
        }
    ]
    
    #streaming_inference(test_messages)
    result.append(capture_inference(test_messages))

<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>John Legend</artist></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>Frederic Chopin</artist><title></title><genre></genre><album></album></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>BTS</artist></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title></title><genre></genre><album></album></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>Jimi Hendrix</artist></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>Bad Bunny</artist><title></title><genre></genre><album></album></arguments></tool_call><end_of_turn><tool_ca

In [12]:
from tqdm import tqdm
from collections import namedtuple

# Определяем именованную структуру
ResponseRecord = namedtuple('ResponseRecord', ['request', 'ground_truth', 'real_response'])

SYSTEM_PROMPT = """You are a tool-using assistant that can understand audio. You are given a user's request in audio format.
You MUST transcribe user's audio and respond with exactly one XML tool call if the tool usage is required (user is asking about play music, asking music title or artist or Genre or Album), otherwise respond to user's request in plain text.
Template: <tool_call><name>audioplay.play_request</name><arguments><artist>ARTIST</artist><title>TITLE</title><genre>GENRE</genre><album>Album</album></arguments></tool_call>. If some arguments is not recognised the correpondence field should be empty. If tool usage is not required respond shortly with text

Also You MUST transcribe user's control command's to stop playing, such as stop, break, cancel to following Template: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

Also You MUST transcribe user's control command's about playing status, such as status (give status) to following Template: <tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>
"""

result = []

for row in tqdm(df.itertuples()):
    text_request = row.Request
    text_gtresponse = row.Response
    
    test_messages = [            
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYSTEM_PROMPT,
                }
            ],
        },
        {
            "role": "user",
            "content": [            
                {"type": "text", "text": text_request}
                
            ]
        }
    ]
    
    # Получаем реальный ответ модели
    print(f"request: {text_request}\n")
    real_response = capture_inference(test_messages)
    print()
    print(f"\nground truth: {text_gtresponse}\n")
    
    # Создаем именованную запись
    record = ResponseRecord(
        request=text_request,
        ground_truth=text_gtresponse,
        real_response=real_response
    )
    
    result.append(record)

0it [00:00, ?it/s]

request: Stop

<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>
<end_of_turn>

1it [00:30, 30.21s/it]



ground truth: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

request: Stop playing

<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>
<end_of_turn>

2it [00:33, 14.11s/it]

...



ground truth: How to meditate for beginners

request: What's happening in the news today?

The user is asking a general question about news. This doesn't require any tool usage.

Response: The user is asking about what's happening in the news today. I am a tool-using assistant designed to handle audio commands related to music playback. I cannot provide news updates.
<end_of_turn>

489it [33:59,  6.00s/it]



ground truth: What's happening in the news today?

request: Summarize today's headlines

Okay, I understand. I will listen to the user's audio request, transcribe it, and respond with either a single XML tool call (if audio playback information is requested) or a plain text response. I will also handle control commands like "stop," "break," or "cancel" and requests for playback status. I will then summarize today's headlines.

Please provide the audio. I'm ready when you are.



<end_of_turn>

490it [34:08,  4.18s/it]



ground truth: Summarize today's headlines



In [14]:
"""
request: Start playing Smells Like Teen Spirit

<tool_call><name>audioplay.play_request</name><arguments><artist>Nirvana</artist><title>Smells Like Teen Spirit</title><genre>Grunge</genre></arguments></tool_call><end_of_turn>
""";

In [15]:
# Сохраняем результат в JSON файл
# Преобразуем namedtuple в словарь для сериализации
import json
result_dicts = [record._asdict() for record in result]

with open('gemma_audio_va_results2.json', 'w', encoding='utf-8') as f:
    json.dump(result_dicts, f, ensure_ascii=False, indent=2)